In [1]:
# General imports
import typing
import random
import os

import pandas as pd
import numpy as np

from simba_ml.simulation import (
    system_model,
    species,
    noisers,
    constraints,
    distributions,
    sparsifier as sparsifier_module,
    kinetic_parameters as kinetic_parameters_module,
    constraints,
    derivative_noiser as derivative_noisers,
    generators
)

In [2]:
name = "MAPK_Signaling"

kinetic_parameters = {
    "compartment": kinetic_parameters_module.ConstantKineticParameter(
        distributions.Constant(1)
    ),  # fixed at 1
    "a_pars": kinetic_parameters_module.ConstantKineticParameter(
        distributions.ContinuousUniformDistribution(800, 1250)
    ),  # sampled between 800 - 1250
    "d_pars": kinetic_parameters_module.ConstantKineticParameter(
        distributions.ContinuousUniformDistribution(100, 225)
    ),  # sampled between 100 - 225
    "k_pars": kinetic_parameters_module.ConstantKineticParameter(
        distributions.ContinuousUniformDistribution(100, 225)
    ),  # sampled between 100 - 225
}


specieses = [
    species.Species(
        str(i), distributions.Constant(0), min_value=0, contained_in_output=False
    )
    for i in range(23)
]
# sample on uniform log scale between e-05 and 3e-04 #E1
specieses[1].distribution = distributions.ContinuousUniformDistribution(1e-5, 3e-4)
# sample on uniform log scale between e-04 and e-03 #E2
specieses[2].distribution = distributions.ContinuousUniformDistribution(1e-4, 1e-3)
# sample between 0.001 and 0.01 #this sets MAPKKK_tot
specieses[3].distribution = distributions.ContinuousUniformDistribution(0.001, 0.01)
# sample between 0.6 and 2.4 #this sets MAPKK_tot
specieses[5].distribution = distributions.ContinuousUniformDistribution(0.6, 2.4)
# sample between 0.6 and 2.4 #this sets MAPK_tot
specieses[8].distribution = distributions.ContinuousUniformDistribution(0.6, 2.4)
# sample between 0.06 and 0.24 #MAPKPase
specieses[11].distribution = distributions.ContinuousUniformDistribution(0.06, 0.24)
# sample between e-04 and e-03 #MAPKKPase
specieses[12].distribution = distributions.ContinuousUniformDistribution(1e-4, 1e-3)

specieses[10].contained_in_output = True
specieses[21].contained_in_output = True
specieses[8].contained_in_output = True

In [6]:
# Define ode system
def func_huang96_biomod(_t: float, x: typing.List[float], arguments: typing.Dict) -> tuple[float, ...]:
    """Defines the derivative of the function at the point _.

    Args:
        _t: (unused) timestep 
        x: Current y vector.
        arguments (dict): Dictionary of arguments configuring the problem.

    Returns:
        Tuple[float, ...]

    This is the MAPK signaling response model from Huang & Ferrell 1996.
    It describes the phosphorylation cascade, and the readout is the amount (percentage)
    of doubly phosphorylated MAPK (K_PP). Its relative abundance is also returned as
    species 23 of the model.
    The function uses the model definition of f() from BIOMD000000000009.m (Octave
    #conversion from SBML file) with adapted definitions of kinetic parameters.
    Compared to the original model, the kinetic parameters are simplified:
    all a-type parameters are identical, all d-type parameters and all k-type parameters.
    """
    # Compartment: id = compartment, name = compartment, constant
    compartment_compartment = arguments["compartment"]  # was 4e-12, can be kept at 1
    # Reaction: id = r1a, name = binding of MAPKKK activator    # Local Parameter:   id =  a1, name = a1
    reaction_r1a_a1 = arguments["a_pars"]
    # Local Parameter:   id =  d1, name = d1
    reaction_r1a_d1 = arguments["d_pars"]
    reaction_r1a = compartment_compartment * (
        reaction_r1a_a1 * x[1] * x[3] - reaction_r1a_d1 * x[13]
    )
    # Reaction: id = r1b, name = MAPKKK activation  # Local Parameter:   id =  k2, name = k2
    reaction_r1b_k2 = arguments["k_pars"]
    reaction_r1b = compartment_compartment * reaction_r1b_k2 * x[13]
    # Reaction: id = r2a, name = binding of MAPKKK inactivator  # Local Parameter:   id =  a2, name = a2
    reaction_r2a_a2 = arguments["a_pars"]
    # Local Parameter:   id =  d2, name = d2
    reaction_r2a_d2 = arguments["d_pars"]
    reaction_r2a = compartment_compartment * (
        reaction_r2a_a2 * x[2] * x[4] - reaction_r2a_d2 * x[14]
    )
    # Reaction: id = r2b, name = MAPKKK inactivation    # Local Parameter:   id =  k2, name = k2
    reaction_r2b_k2 = arguments["k_pars"]
    reaction_r2b = compartment_compartment * reaction_r2b_k2 * x[14]
    # Reaction: id = r3a, name = binding P-MAPKKK and MAPKK # Local Parameter:   id =  a3, name = a3
    reaction_r3a_a3 = arguments["a_pars"]
    # Local Parameter:   id =  d3, name = d3
    reaction_r3a_d3 = arguments["d_pars"]
    reaction_r3a = compartment_compartment * (
        reaction_r3a_a3 * x[5] * x[4] - reaction_r3a_d3 * x[15]
    )
    # Reaction: id = r3b, name = phosphorylation of MAPKK   # Local Parameter:   id =  k3, name = k3
    reaction_r3b_k3 = arguments["k_pars"]
    reaction_r3b = compartment_compartment * reaction_r3b_k3 * x[15]
    # Reaction: id = r4a, name = binding MAPKK-Pase and P-MAPKK # Local Parameter:   id =  a4, name = a4
    reaction_r4a_a4 = arguments["a_pars"]
    # Local Parameter:   id =  d4, name = d4
    reaction_r4a_d4 = arguments["d_pars"]
    reaction_r4a = compartment_compartment * (
        reaction_r4a_a4 * x[6] * x[12] - reaction_r4a_d4 * x[20]
    )
    # Reaction: id = r4b, name = dephosphorylation of P-MAPKK   # Local Parameter:   id =  k4, name = k4
    reaction_r4b_k4 = arguments["k_pars"]
    reaction_r4b = compartment_compartment * reaction_r4b_k4 * x[20]
    # Reaction: id = r5a, name = binding P-MAPKKK and P-MAPKK   # Local Parameter:   id =  a5, name = a5
    reaction_r5a_a5 = arguments["a_pars"]
    # Local Parameter:   id =  d5, name = d5
    reaction_r5a_d5 = arguments["d_pars"]
    reaction_r5a = compartment_compartment * (
        reaction_r5a_a5 * x[6] * x[4] - reaction_r5a_d5 * x[16]
    )
    # Reaction: id = r5b, name = phosphorylation of P-MAPKK # Local Parameter:   id =  k5, name = k5
    reaction_r5b_k5 = arguments["k_pars"]
    reaction_r5b = compartment_compartment * reaction_r5b_k5 * x[16]
    # Reaction: id = r6a, name = binding MAPKK-Pase and PP-MAPKK    # Local Parameter:   id =  a6, name = a6
    reaction_r6a_a6 = arguments["a_pars"]
    # Local Parameter:   id =  d6, name = d6
    reaction_r6a_d6 = arguments["d_pars"]
    reaction_r6a = compartment_compartment * (
        reaction_r6a_a6 * x[7] * x[12] - reaction_r6a_d6 * x[19]
    )
    # Reaction: id = r6b, name = dephosphorylation of PP-MAPKK  # Local Parameter:   id =  k6, name = k6
    reaction_r6b_k6 = arguments["k_pars"]
    reaction_r6b = compartment_compartment * reaction_r6b_k6 * x[19]
    # Reaction: id = r7a, name = binding MAPK and PP-MAPKK  # Local Parameter:   id =  a7, name = a7
    reaction_r7a_a7 = arguments["a_pars"]
    # Local Parameter:   id =  d7, name = d7
    reaction_r7a_d7 = arguments["d_pars"]
    reaction_r7a = compartment_compartment * (
        reaction_r7a_a7 * x[8] * x[7] - reaction_r7a_d7 * x[17]
    )
    # Reaction: id = r7b, name = phosphorylation of MAPK    # Local Parameter:   id =  k7, name = k7
    reaction_r7b_k7 = arguments["k_pars"]
    reaction_r7b = compartment_compartment * reaction_r7b_k7 * x[17]
    # Reaction: id = r8a, name = binding MAPK-Pase and P-MAPK   # Local Parameter:   id =  a8, name = a8
    reaction_r8a_a8 = arguments["a_pars"]
    # Local Parameter:   id =  d8, name = d8
    reaction_r8a_d8 = arguments["d_pars"]
    reaction_r8a = compartment_compartment * (
        reaction_r8a_a8 * x[9] * x[11] - reaction_r8a_d8 * x[22]
    )
    # Reaction: id = r8b, name = dephosphorylation of P-MAPK    # Local Parameter:   id =  k8, name = k8
    reaction_r8b_k8 = arguments["k_pars"]
    reaction_r8b = compartment_compartment * reaction_r8b_k8 * x[22]
    # Reaction: id = r9a, name = binding PP-MAPKK and P-MAPK    # Local Parameter:   id =  a9, name = a9
    reaction_r9a_a9 = arguments["a_pars"]
    # Local Parameter:   id =  d9, name = d9
    reaction_r9a_d9 = arguments["d_pars"]
    reaction_r9a = compartment_compartment * (
        reaction_r9a_a9 * x[9] * x[7] - reaction_r9a_d9 * x[18]
    )
    # Reaction: id = r9b, name = phosphorylation of P-MAPK  # Local Parameter:   id =  k9, name = k9
    reaction_r9b_k9 = arguments["k_pars"]
    reaction_r9b = compartment_compartment * reaction_r9b_k9 * x[18]
    # Reaction: id = r10a, name = binding MAPK-Pase and PP-MAPK # Local Parameter:   id =  a10, name = a10
    reaction_r10a_a10 = arguments["a_pars"]
    # Local Parameter:   id =  d10, name = d10
    reaction_r10a_d10 = arguments["d_pars"]
    reaction_r10a = compartment_compartment * (
        reaction_r10a_a10 * x[10] * x[11] - reaction_r10a_d10 * x[21]
    )
    # Reaction: id = r10b, name = dephosphorylation of PP-MAPK  # Local Parameter:   id =  k10, name = k10
    reaction_r10b_k10 = arguments["k_pars"]
    reaction_r10b = compartment_compartment * reaction_r10b_k10 * x[21]
    xdot = np.zeros((23, 1))
    # Species:   id = E1, name = MAPKKK activator (Ras), affected by kineticLaw
    xdot[1] = (1 / (compartment_compartment)) * (
        (-1.0 * reaction_r1a) + (1.0 * reaction_r1b)
    )
    # Species:   id = E2, name = MAPKKK inactivator, affected by kineticLaw
    xdot[2] = (1 / (compartment_compartment)) * (
        (-1.0 * reaction_r2a) + (1.0 * reaction_r2b)
    )
    # Species:   id = KKK, name = Mos, affected by kineticLaw
    xdot[3] = (1 / (compartment_compartment)) * (
        (-1.0 * reaction_r1a) + (1.0 * reaction_r2b)
    )
    # Species:   id = P_KKK, name = Mos-P, affected by kineticLaw
    xdot[4] = (1 / (compartment_compartment)) * (
        (1.0 * reaction_r1b)
        + (-1.0 * reaction_r2a)
        + (-1.0 * reaction_r3a)
        + (1.0 * reaction_r3b)
        + (-1.0 * reaction_r5a)
        + (1.0 * reaction_r5b)
    )
    # Species:   id = KK, name = Mek1, affected by kineticLaw
    xdot[5] = (1 / (compartment_compartment)) * (
        (-1.0 * reaction_r3a) + (1.0 * reaction_r4b)
    )
    # Species:   id = P_KK, name = Mek1-P, affected by kineticLaw
    xdot[6] = (1 / (compartment_compartment)) * (
        (1.0 * reaction_r3b)
        + (-1.0 * reaction_r4a)
        + (-1.0 * reaction_r5a)
        + (1.0 * reaction_r6b)
    )
    # Species:   id = PP_KK, name = Mek1-PP, affected by kineticLaw
    xdot[7] = (1 / (compartment_compartment)) * (
        (1.0 * reaction_r5b)
        + (-1.0 * reaction_r6a)
        + (-1.0 * reaction_r7a)
        + (1.0 * reaction_r7b)
        + (-1.0 * reaction_r9a)
        + (1.0 * reaction_r9b)
    )
    # Species:   id = K, name = Erk2, affected by kineticLaw
    xdot[8] = (1 / (compartment_compartment)) * (
        (-1.0 * reaction_r7a) + (1.0 * reaction_r8b)
    )
    # Species:   id = P_K, name = Erk2-P, affected by kineticLaw
    xdot[9] = (1 / (compartment_compartment)) * (
        (1.0 * reaction_r7b)
        + (-1.0 * reaction_r8a)
        + (-1.0 * reaction_r9a)
        + (1.0 * reaction_r10b)
    )
    # Species:   id = PP_K, name = Erk2-PP, affected by kineticLaw
    xdot[10] = (1 / (compartment_compartment)) * (
        (1.0 * reaction_r9b) + (-1.0 * reaction_r10a)
    )
    # Species:   id = KPase, name = MAPK-Pase, affected by kineticLaw
    xdot[11] = (1 / (compartment_compartment)) * (
        (-1.0 * reaction_r8a)
        + (1.0 * reaction_r8b)
        + (-1.0 * reaction_r10a)
        + (1.0 * reaction_r10b)
    )
    # Species:   id = KKPase, name = MAPKK-Pase, affected by kineticLaw
    xdot[12] = (1 / (compartment_compartment)) * (
        (-1.0 * reaction_r4a)
        + (1.0 * reaction_r4b)
        + (-1.0 * reaction_r6a)
        + (1.0 * reaction_r6b)
    )
    # Species:   id = E1_KKK, name = E1_Mos, affected by kineticLaw
    xdot[13] = (1 / (compartment_compartment)) * (
        (1.0 * reaction_r1a) + (-1.0 * reaction_r1b)
    )
    # Species:   id = E2_P_KKK, name = E2_Mos-P, affected by kineticLaw
    xdot[14] = (1 / (compartment_compartment)) * (
        (1.0 * reaction_r2a) + (-1.0 * reaction_r2b)
    )
    # Species:   id = P_KKK_KK, name = P-Mos_Mek1, affected by kineticLaw
    xdot[15] = (1 / (compartment_compartment)) * (
        (1.0 * reaction_r3a) + (-1.0 * reaction_r3b)
    )
    # Species:   id = P_KKK_P_KK, name = P-Mos_P-Mek1, affected by kineticLaw
    xdot[16] = (1 / (compartment_compartment)) * (
        (1.0 * reaction_r5a) + (-1.0 * reaction_r5b)
    )
    # Species:   id = PP_KK_K, name = PP-Mek1_Erk2, affected by kineticLaw
    xdot[17] = (1 / (compartment_compartment)) * (
        (1.0 * reaction_r7a) + (-1.0 * reaction_r7b)
    )
    # Species:   id = PP_KK_P_K, name = PP-Mek1_P-Erk2, affected by kineticLaw
    xdot[18] = (1 / (compartment_compartment)) * (
        (1.0 * reaction_r9a) + (-1.0 * reaction_r9b)
    )
    # Species:   id = KKPase_PP_KK, name = MAPKK-Pase_PP-Mek1, affected by kineticLaw
    xdot[19] = (1 / (compartment_compartment)) * (
        (1.0 * reaction_r6a) + (-1.0 * reaction_r6b)
    )
    # Species:   id = KKPase_P_KK, name = MAPKK-Pase_P-Mek1, affected by kineticLaw
    xdot[20] = (1 / (compartment_compartment)) * (
        (1.0 * reaction_r4a) + (-1.0 * reaction_r4b)
    )
    # Species:   id = KPase_PP_K, name = MAPK-Pase_PP-Erk2, affected by kineticLaw
    xdot[21] = (1 / (compartment_compartment)) * (
        (1.0 * reaction_r10a) + (-1.0 * reaction_r10b)
    )
    # Species:   id = KPase_P_K, name = MAPK-Pase_P-Erk2, affected by kineticLaw
    xdot[22] = (1 / (compartment_compartment)) * (
        (1.0 * reaction_r8a) + (-1.0 * reaction_r8b)
    )
    return xdot

In [7]:
# Add lognormal distributed noise to the data
noiser = noisers.ColumnNoiser(
    {
        "10": noisers.MultiplicativeNoiser(
            distributions.LogNormalDistribution(0, 0.05)
        ),
        "21": noisers.MultiplicativeNoiser(
            distributions.LogNormalDistribution(0, 0.05)
        ),
    }
)

In [8]:
# Create a systemmodel to represent the system
sm = system_model.SystemModel(
    name,
    specieses,
    kinetic_parameters,
    noiser=noiser,
    deriv=func_huang96_biomod,
    timestamps=distributions.Constant(20),
)

In [9]:
# Generate time-series
number_of_time_series = [100, 1_000, 10_000]
for n in number_of_time_series:
    generators.TimeSeriesGenerator(sm).generate_csvs(n, f"./simulated_data_with_noise_{n}/")

In [10]:
# Transform generated data
for n in number_of_time_series:
    input_datafolder = f"./simulated_data_with_noise_{n}/"
    output_datafolder = f"./transformed_data_with_noise_{n}/"

    if not os.path.exists(input_datafolder):
        raise ValueError("Input data folder does not exist.")

    if not os.path.exists(output_datafolder):
        os.makedirs(output_datafolder)

    for file in os.listdir(input_datafolder):
        if not file.endswith(".csv"):
            continue
        df = pd.read_csv(os.path.join(input_datafolder, file))
        df["combined"] = (df["10"] + df["21"]) / df["8"].iloc[0]
        df[["combined"]].to_csv(os.path.join(output_datafolder, file), index=False)

In [12]:
# Run machine learning pipeline
from simba_ml.prediction.time_series.pipelines import synthetic_data_pipeline
results = []
for n in number_of_time_series:
    results_df = synthetic_data_pipeline.main(f"ml_config_{n}.toml")
    results.append(results_df)

UnexpectedDataError: can not match "num_species" to any data class field